In [3]:
import json
import requests

In [6]:
# Looking into starwars api, start at the top and build dictionaries for each schema:
baseurl = 'https://swapi.dev/api/'
base_dict = requests.get(baseurl).json()
print(f'These are the major schema in the starwars API:\n {base_dict.keys()}')

These are the major schema in the starwars API:
 dict_keys(['people', 'planets', 'films', 'species', 'vehicles', 'starships'])


In [9]:
base_dict.items()

dict_items([('people', 'https://swapi.dev/api/people/'), ('planets', 'https://swapi.dev/api/planets/'), ('films', 'https://swapi.dev/api/films/'), ('species', 'https://swapi.dev/api/species/'), ('vehicles', 'https://swapi.dev/api/vehicles/'), ('starships', 'https://swapi.dev/api/starships/')])

In [61]:
def get_info(response, info_bit):
    return response[info_bit]

def explore_api(head):
    """ Gets the page info for each schema in the 
    primary dictionary of the API
    
    - Accepts the primary dictionary
    - prints and returns dict_explore with keys: schema, 
                                        values: [endpoints,counts] """
    dict_explore = {}
    print('Schema, Endpoint, Number of pages')
    for schema, endpoint in head.items():
        dict_schema = requests.get(endpoint).json()
        count = get_info(dict_schema,"count")
        print(schema, 
              endpoint, 
              f'Number of counts: {count}')
        dict_explore.update({schema:[endpoint, count]})
    return dict_explore

explore = explore_api(base_dict)

Schema, Endpoint, Number of pages
people https://swapi.dev/api/people/ Number of counts: 82
planets https://swapi.dev/api/planets/ Number of counts: 60
films https://swapi.dev/api/films/ Number of counts: 6
species https://swapi.dev/api/species/ Number of counts: 37
vehicles https://swapi.dev/api/vehicles/ Number of counts: 39
starships https://swapi.dev/api/starships/ Number of counts: 36


In [62]:
explore

{'people': ['https://swapi.dev/api/people/', 82],
 'planets': ['https://swapi.dev/api/planets/', 60],
 'films': ['https://swapi.dev/api/films/', 6],
 'species': ['https://swapi.dev/api/species/', 37],
 'vehicles': ['https://swapi.dev/api/vehicles/', 39],
 'starships': ['https://swapi.dev/api/starships/', 36]}

In [66]:
def get_schema_allpages(endpoint, count):
    dict_allpages = {}    
    response = requests.get(endpoint+f'?page=1').json()
    lst_results = response['results']
    while response['next']:
        response = requests.get(response['next']).json()
        lst_results.extend(response['results'])       
    dict_allpages['results'] = lst_results   
    # return dictionary of all the results
    return dict_allpages

In [75]:
query = explore['people']
dict_schema = get_schema_allpages(query[0],query[1])
#big_dict = get_whole_dict(base_dict)

In [79]:
list_obj = dict_schema['results']

In [80]:
list_obj


[{'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/2/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['https://swapi.dev/api/vehicles/14/',
   'https://swapi.dev/api/vehicles/30/'],
  'starships': ['https://swapi.dev/api/starships/12/',
   'https://swapi.dev/api/starships/22/'],
  'created': '2014-12-09T13:50:51.644000Z',
  'edited': '2014-12-20T21:17:56.891000Z',
  'url': 'https://swapi.dev/api/people/1/'},
 {'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
 

### GOALS:
1. Create a function that returns a list of all the names.
2. Create a function that counts the total number of males and females. Return both numbers
3. Create a function that returns a list of all the charachters that appear in more than x number of films. Where 'x' is the number of films you are checking for and is passed into the function.
4. Create function that returns a tuple of the min, max, and avg height for all charachters
5. Create a function that accepts two arguements: eye color and hair color. Return a dictionary with the min, max, and avg height based on the parameters.

* Challenge 1: Get all of the response data into a list using a "While Loop" instead of a "For Loop"
* Challenge 2: For problem number 4, convert the centimeters to feet

In [84]:
def get_names_list(list_obj):
    names = []
    for diction in list_obj:
        names.append(dict.get('name'))
    return names

In [88]:
names = get_names_list(list_obj)
assert len(names) == explore['people'][1]

In [185]:
def genders(list_obj):
    males = 0
    females = 0
    others = []
    for d in list_obj:
        gender = d['gender']
        if gender == 'male':
            males += 1
        elif gender == 'female':
            females += 1
        else:
            others.append(gender)
    return males, females, others

In [186]:
genders(list_obj)

(60, 17, ['n/a', 'n/a', 'n/a', 'hermaphrodite', 'none'])

### we can also append all genders and use Counter on them.

In [97]:
from collections import Counter
def genders_counter(list_obj):
    genders = []
    for d in list_obj:
        genders.append(d.get('gender'))
    return Counter(genders)

In [100]:
genders_dict = genders_counter(list_obj)
assert sum([val for key, val in genders_dict.items()]) == explore['people'][1]

### 3. Create a function that returns a list of all the charachters that appear in more than x number of films. Where 'x' is the number of films you are checking for and is passed into the function.

In [106]:
# lets use namedtuple
def get_characters_appearing(list_obj, n):
    appearance = {}
    for dict_element in list_obj:
        appearance[dict_element['name']] = len(dict_element['films'])
    list_chars = [key for key, val in appearance.items() if val > n]  
    return list_chars

In [113]:
get_characters_appearing(list_obj, n=5)

['C-3PO', 'R2-D2', 'Obi-Wan Kenobi']

### 4. Create function that returns a tuple of the min, max, and avg height for all charachters

In [175]:
def weight_stats(list_obj, feet=False):
    height = []
    for dict_element in list_obj:
        if dict_element['height'] != 'unknown':
            height.append(int(dict_element['height']))
    if feet:
        cm2feet = 0.03280839895
        height = list(map(lambda x: x * cm2feet, height))
    return min(height), max(height), round(sum(height)/len(height),2)
        

In [176]:
weight_stats(list_obj, feet=True)

(2.1653543307, 8.6614173228, 5.73)

### 5. Create a function that accepts two arguements: eye color and hair color. Return a dictionary with the min, max, and avg height based on the parameters.

In [151]:
def weight_stats(list_obj, eye_color, hair_color):

    height = []
    for dict_element in list_obj:
        if dict_element['height'] != 'unknown'\
            and dict_element['hair_color'] == hair_color\
            and dict_element['eye_color'] == eye_color:
            
            print(dict_element['name'],int(dict_element['height']))
            height.append(int(dict_element['height']) )
    
    if height:
        return min(height), max(height), round(sum(height)/len(height),2)
    else:
        return None

In [149]:
weight_stats(list_obj, eye_color='blue', hair_color='blond')

Luke Skywalker 172
Anakin Skywalker 188
Finis Valorum 170


(170, 188, 176.67)

In [180]:
query = explore['films']
dict_schema_films = get_schema_allpages(query[0],query[1])
films = dict_schema_films['results']

In [184]:
men = ['Luke Skywalker', 'Anakin Skywalker', 'Darth Vader']
for m in men:
    for dict_element in list_obj:
        if dict_element['name'] == m:
            film_api = dict_element['films']#.split('/')
            print(film_api)

['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/']
['https://swapi.dev/api/films/4/', 'https://swapi.dev/api/films/5/', 'https://swapi.dev/api/films/6/']
['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/']


In [181]:
films = dict_schema_films['results']

In [182]:
films

[{'title': 'A New Hope',
  'episode_id': 4,
  'opening_crawl': "It is a period of civil war.\r\nRebel spaceships, striking\r\nfrom a hidden base, have won\r\ntheir first victory against\r\nthe evil Galactic Empire.\r\n\r\nDuring the battle, Rebel\r\nspies managed to steal secret\r\nplans to the Empire's\r\nultimate weapon, the DEATH\r\nSTAR, an armored space\r\nstation with enough power\r\nto destroy an entire planet.\r\n\r\nPursued by the Empire's\r\nsinister agents, Princess\r\nLeia races home aboard her\r\nstarship, custodian of the\r\nstolen plans that can save her\r\npeople and restore\r\nfreedom to the galaxy....",
  'director': 'George Lucas',
  'producer': 'Gary Kurtz, Rick McCallum',
  'release_date': '1977-05-25',
  'characters': ['https://swapi.dev/api/people/1/',
   'https://swapi.dev/api/people/2/',
   'https://swapi.dev/api/people/3/',
   'https://swapi.dev/api/people/4/',
   'https://swapi.dev/api/people/5/',
   'https://swapi.dev/api/people/6/',
   'https://swapi.dev/ap